# CS 4650 - Natural Language Understanding - HW - 2 
Georgia Tech, Spring 2024 (Instructor: Kartik Goyal)

Welcome to the last full programming assignment for CS 4650! 

In this assignment, you will be implementing a Neural Machine Translation model for a TED Talks transcript dataset.

You are expected to have a good understanding of NumPy and PyTorch before starting this assignment.

- NumPy Quickstarter Guide: https://numpy.org/doc/stable/user/quickstart.html
- A good tutorial on PyTorch: https://www.youtube.com/watch?v=OIenNRt2bjg
- Detailed Documentation of PyTorch: https://pytorch.org/docs/stable/index.html

<font color='red'> DEADLINE: April 24, 2024, 11:59 PM  </font><br>

Please note the following -
- We believe that this assignment will require a considerable effort. So, please start early.
- The Gradescope tests might take time to run. Therefore, as much as possible, use local tests to debug your code. We have provided similar local test cases for most of the methods.

Please note the following points breakup of the assignment:

| **Section** | **Topic**                | **Points** |
|-------------|--------------------------|------------|
| 1           | Load and Preprocess Data | 3          |
| 2           | NMT Model                | 48         |
| 3           | Sampling                 | 34         |
| 4           | BONUS                    | 15         |


**To start, first make a copy of this notebook to your local drive, so you can edit it.**

## 0. Colab Setup

Note - It is heavily recommended you utilize high-end GPUs (A100/V100) of Colab Pro for this assignment. However, be smart to not finish your compute units. You don't need GPUs to test the forward passes and unit tests of your models. It is only needed to train your model and calculate BLEU scores (where forward pass on the entire dataset is needed).

In [ ]:
!pip install nltk
!pip install tqdm

# Add any other required packages here

Next, upload the entire directory to your Google drive and do the following -

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "/content/drive/MyDrive/.../directory_of_this_amazing_NLP_hw" # Change this to the directory where you have the files of the homework

In [1]:
%load_ext autoreload
%autoreload 2

## 1. Load and Preprocess Data [Programming] [3 points]

In this section, you will load and preprocess the data for the Neural Machine Translation model. The data is in the form of parallel sentences in two languages - German and English. The data is already tokenized and lowercased.

You will need to implement the following functions:
1. `build` in the `Vocab` class - 3 points

In [17]:
# DO NOT CHANGE THIS CODE BLOCK

from vocab import Vocab
from utils import read_corpus, set_model_weights
from nmt_model import NMT
from model_training import train
from tqdm import tqdm

import torch
import numpy as np

import random

In [18]:
# DO NOT CHANGE THIS CODE BLOCK

language_mapper = {
    'english': 'en',
    'german': 'de'
}

In [19]:
# DO NOT CHANGE THIS CODE BLOCK

SOURCE = 'german'
TARGET = 'english'
VOCAB_SIZE = 30000
FREQUENCY_CUTOFF = 2
RANDOM_SEED = 42
CLIP_GRAD = 5.0
LOG_EVERY = 10000
VALID_NITER = 2400

torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
device = torch.device('cuda' if torch.cuda.is_available() else('mps' if torch.backends.mps.is_available() else 'cpu'))
if device == 'cuda':
    torch.cuda.manual_seed(RANDOM_SEED)

In [20]:
# Read the training corpus
train_src = f'./data/train.{language_mapper[SOURCE]}-{language_mapper[TARGET]}.{language_mapper[SOURCE]}.wmixerprep'
train_tgt = f'./data/train.{language_mapper[SOURCE]}-{language_mapper[TARGET]}.{language_mapper[TARGET]}.wmixerprep'

# Read the validation corpus
valid_src = f"./data/valid.{language_mapper[SOURCE]}-{language_mapper[TARGET]}.{language_mapper[SOURCE]}"
valid_tgt = f"./data/valid.{language_mapper[SOURCE]}-{language_mapper[TARGET]}.{language_mapper[TARGET]}"

In [ ]:
# DO NOT CHANGE THIS CODE BLOCK

src_sentences = read_corpus(train_src, source='src')
tgt_sentences = read_corpus(train_tgt, source='tgt')

vocab = Vocab.build(src_sentences, tgt_sentences, VOCAB_SIZE, FREQUENCY_CUTOFF)
print('Generated Vocabulary\nSource: %d words\nTarget: %d words' % (len(vocab.src), len(vocab.tgt)))

assert len(vocab.src) == 30003, "Training source vocabulary is of incorrect size"
assert len(vocab.tgt) == 22824, "Training target vocabulary is of incorrect size"

print("All tests passed")

vocab.save("./vocab.json")

In [22]:
# DO NOT CHANGE THIS CODE BLOCK

train_data_src = read_corpus(train_src, source='src')
train_data_tgt = read_corpus(train_tgt, source='tgt')

valid_data_src = read_corpus(valid_src, source='src')
valid_data_tgt = read_corpus(valid_tgt, source='tgt')

train_data = list(zip(train_data_src, train_data_tgt))
valid_data = list(zip(valid_data_src, valid_data_tgt))

vocab = Vocab.load('./vocab.json')

In [23]:
# Utility Functions - DO NOT CHANGE
from utils import read_corpus
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu
import sys
TEST_SOURCE_FILE = "data/test.de-en.de"
TEST_TARGET_FILE = "data/test.de-en.en"
def beam_search(model, test_data_src, beam_size, max_decoding_time_step):
    was_training = model.training
    model.eval()

    hypotheses = []
    with torch.no_grad():
        for src_sent in tqdm(test_data_src, desc='Decoding'):
            example_hyps = model.beam_search(src_sent, beam_size=beam_size, max_decoding_time_step=max_decoding_time_step)

            hypotheses.append(example_hyps)

    if was_training: model.train(was_training)

    return hypotheses
def compute_corpus_level_bleu_score(references, hypotheses):
    """
    Given decoding results and reference sentences, compute corpus-level BLEU score

    Args:
        references: a list of gold-standard reference target sentences
        hypotheses: a list of hypotheses, one for each reference

    Returns:
        bleu_score: corpus-level BLEU score
    """

    if references[0][0] == '<s>':
        references = [ref[1:-1] for ref in references]

    bleu_score = corpus_bleu([[ref] for ref in references],
                             [hyp.value for hyp in hypotheses])

    return bleu_score * 100

def decode(model, beam_size=5, max_decoding_time_step=100, device='cuda'):
    """
    Performs decoding on a test set, and save the best-scoring decoding results.
    If the target gold-standard sentences are given, the function also computes
    corpus-level BLEU score.
    """


    # for the purposes of this assignment, we will evaluate bleu over the val set, where normally you would look over a test set.
    test_data_src = valid_data_src
    test_data_tgt = valid_data_tgt

    model = model.to(device)

    hypotheses = beam_search(model, test_data_src,
                             beam_size=int(beam_size),
                             max_decoding_time_step=int(max_decoding_time_step))

    top_hypotheses = [hyps[0] for hyps in hypotheses]
    bleu_score = compute_corpus_level_bleu_score(test_data_tgt, top_hypotheses)
    print(f'Corpus BLEU: {bleu_score}')


## 2. NMT Model [Programming + Non-Programming] [48 points]

In this section, you will implement the Neural Machine Translation model. The model is based on the paper "Effective Approaches to Attention-based Neural Machine Translation" by Luong et al. (2015). The paper can be found in the following [link](https://arxiv.org/abs/1508.04025). The model that is being implemented is an LSTM encoder decoder, which is augmented by attention for the decoder. The model is implemented in the `NMT` class in the `nmt_model.py` file.

You will need to implement the following functions in the `NMT` class in the `nmt_model.py` file:
1. `__init__` and `forward` - 3 points
2. `encode` - 6 points
3. `dot_prod_attention` - 6 points
4. `step` - 6 points
5. `decode` - 6 points
6. `beam_search` - 3 points

You will need to implement the following functions in the `LabelSmoothingLoss` in the `utils.py` file:
1. `__init__` and `forward` - 6 points

Apart from this, the following parts should be done in the notebook:
1. Reporting BLEU score - 6 points
2. Trying different Label Smoothing variations - 6 points

The description for each function is provided in the respective files. Please read the description carefully before implementing the functions.

The following tests for the above implementation does not require cuda, and can be run locally to sanity check your model implementation. We will initialize a dummy model for this purpose.

A recommendation to get started implementing the functions is to read through the paper for which this model definition is based off of, and understand the equations (this could take about 4 hours if read thoroughly with proper note taking), then proceed with implementation. We have removed input_feeding from the model for a minor simplificaiton from the paper's global attention model.

At this point the forward function local test should have the provided return value, and you are expected to be able to train a label smoothing enabled model from scratch! The expected validation metric for this model on decoding is 8.81 ppl during training eval on the dev set, which is used to evaluate early stopping (This is with the default settings --batch-size 64 --hidden-size 256 --embed-size 256 --uniform-init 0.1 --label-smoothing 0.1 --dropout 0.2, --clip-grad 5.0 --lr-decay 0.5). 

In [40]:
# DO NOT CHANGE THIS CODE BLOCK

model_dummy = NMT(embed_size=256, hidden_size=256, dropout_rate=0.2, vocab=vocab, label_smoothing=0.1)
set_model_weights(model_dummy)

In [ ]:
# DO NOT CHANGE THIS CODE BLOCK

model_dummy.eval() # this is so the model is deterministic with the dropout
input = "hello there ! in german".split(" ")
target = ['<s>'] + "hello there ! in english".split(" ") + ['</s>'] + ["<pad>"]
print("input", input)
print("target", target)
src_sents = [input, input[2:]]
tgt_sents = [target, ["<s>"] + target[2:]]

### 2.1 Initialization and Forward Pass [Programming] (3 points)

The forward function in the NMT class is the main function that is called when the model is run. It takes in the source and target sentences and returns the loss. The forward function is implemented in the `NMT` class in the `nmt_model.py` file.

In [ ]:
print(model_dummy.forward(src_sents, tgt_sents))
print("Expected:", torch.tensor([-454.5613, -341.4756]))

print("All tests passed!")

### 2.2 Encoder [Programming] (6 points)

The encoder is a bidirectional LSTM that takes in the source sentence and produces a context vector for the decoder. The encoder is implemented in the `encode` function in the `NMT` class in the `nmt_model.py` file.

In [ ]:
src_sents_var = model_dummy.vocab.src.to_input_tensor(src_sents, device=model_dummy.device)
tgt_sents_var = model_dummy.vocab.tgt.to_input_tensor(tgt_sents, device=model_dummy.device)
src_sents_len = [len(s) for s in src_sents]

src_encodings, decoder_init_vec = model_dummy.encode(src_sents_var, src_sents_len)
src_sent_masks = model_dummy.get_attention_mask(src_encodings, src_sents_len)
(decoder_init_state, decoder_init_cell) = decoder_init_vec

assert src_encodings.shape == torch.Size([2, 5, 512]), f"Incorrect shape for src_encodings {src_encodings.shape}"
assert decoder_init_state.shape == torch.Size([2, 256]), f"Incorrect shape for decoder_init_state {decoder_init_state.shape}"
assert decoder_init_cell.shape == torch.Size([2, 256]), f"Incorrect shape for decoder_init_cell {decoder_init_cell.shape}"
assert abs(src_encodings[0,0,0] - -3.3668e-05) < 0.001, f"Incorrect first entry in src_encodings {src_encodings[0,0,0]}"
assert abs(src_encodings[-1,-2,-1] - 0) < 0.001, f"Incorrect second to last entry in src_encodings {src_encodings[-1,-2,-1]}"
assert abs(src_encodings[-1,-3,-1] - -0.7592) < 0.001, f"Incorrect third last entry in src_encodings {src_encodings[-1,-3,-1]}"
assert abs(decoder_init_state[0,0] - 1) < 0.001, f"Incorrect first entry in decoder_init_state {decoder_init_state[0,0]}"
assert abs(decoder_init_state[-1,-5] - 0.9273) < 0.001, f"Incorrect fifth last entry in decoder_init_state {decoder_init_state[-1,-5]}"
assert abs(decoder_init_cell[0,0] - 25.2386) < 0.001, f"Incorrect first entry in decoder_init_cell {decoder_init_cell[0,0]}"
assert abs(decoder_init_cell[-1,-1] - 30.4197) < 0.001, f"Incorrect last entry in decoder_init_cell {decoder_init_cell[-1,-1]}"

print("All tests passed!")

### 2.3 Attention [Programming] (6 points)

The attention mechanism is used to focus on different parts of the source sentence at each step of the decoding process. The attention mechanism is implemented in the `dot_prod_attention` function in the `NMT` class in the `nmt

In [ ]:
ctx_t, alpha_t = model_dummy.dot_prod_attention(h_t=torch.ones((2, 256)).float(), src_encoding=src_encodings, src_encoding_att_linear=torch.ones((2, 5, 256)).float(), mask=src_sent_masks)

assert ctx_t.shape == torch.Size([2, 512]), f"Incorrect shape for ctx_t: {ctx_t.shape}"
assert alpha_t.shape == torch.Size([2, 5]), f"Incorrect shape for alpha_t: {alpha_t.shape}"
assert abs(ctx_t[0, 0].item() - 0.1514) < 0.001, f"Incorrect value for ctx_t[0, 0]: {ctx_t[0, 0].item()}"
assert abs(alpha_t[0, 0].item() - 0.2000) < 0.001, f"Incorrect value for alpha_t[0, 0]: {alpha_t[0, 0].item()}"
assert abs(ctx_t[-1, -1].item() - (-0.2546)) < 0.001, f"Incorrect value for ctx_t[-1, -1]: {ctx_t[-1, -1].item()}"
assert abs(alpha_t[-1, -1].item() - 0.0) < 0.001, f"Incorrect value for alpha_t[-1, -1]: {alpha_t[-1, -1].item()}"
assert abs(alpha_t[-1, 0].item() - 0.3333) < 0.001, f"Incorrect value for alpha_t[-1, 0]: {alpha_t[-1, 0].item()}"

print("All tests passed!")

### 2.4 Decoder Step [Programming] (6 points)

The decoder step is the core of the decoder, where the decoder LSTM cell is run for one step. The decoder step is implemented in the `step` function in the `NMT` class in the `nmt_model.py` file.

In [ ]:
(h_t, cell_t), att_t, alpha_t = model_dummy.step(x=torch.arange(2*256).reshape(2, 256) / 512, h_tm1=decoder_init_vec, src_encodings=src_encodings, src_encoding_att_linear=torch.arange(2*5*256).reshape(2, 5, 256) / (2 * 5 * 256), src_sent_masks=src_sent_masks)

assert h_t.shape == torch.Size([2, 256]), f"Incorrect shape for h_t: {h_t.shape}"
assert cell_t.shape == torch.Size([2, 256]), f"Incorrect shape for cell_t: {cell_t.shape}"
assert att_t.shape == torch.Size([2, 256]), f"Incorrect shape for att_t: {att_t.shape}"
assert alpha_t.shape == torch.Size([2, 5]), f"Incorrect shape for alpha_t: {alpha_t.shape}"
assert abs(h_t[0, 1].item() - (-0.9392)) < 0.001, f"Incorrect value for the second element of h_t: {h_t[0, 1].item()}"
assert abs(h_t[-1,-2].item() - 0.0155) < 0.001, f"Incorrect value for the second to last element of h_t: {h_t[-1, -2].item()}"
assert abs(cell_t[0, 0].item() - 25.2386) < 0.001, f"Incorrect value for the first element of cell_t: {cell_t[0, 0].item()}"
assert abs(cell_t[-1, -1].item() - (31.4197)) < 0.001, f"Incorrect value for the last element of cell_t: {cell_t[-1, -1].item()}"
assert abs(att_t[0, 1].item() - 0.9978) < 0.001, f"Incorrect value for the second element of att_t: {att_t[0, 1].item()}"
assert abs(att_t[-1, -3].item() - (-0.9195)) < 0.001, f"Incorrect value for the third to last element of att_t: {att_t[-1, -3].item()}"
assert abs(alpha_t[0, 0].item() - 0.1153) < 0.001, f"Incorrect value for the first element of alpha_t: {alpha_t[0, 0].item()}"
assert abs(alpha_t[-1, -3].item() - 0.1293) < 0.001, f"Incorrect value for the third to last element of alpha_t: {alpha_t[-1, -3].item()}"

print("All tests passed!")

### 2.5 Decoder [Programming] (6 points)

The decoder is an LSTM that takes in the context vector from the encoder and produces the output sequence. The decoder is implemented in the `decode` function in the `NMT` class in the `nmt_model.py` file.

In [ ]:
att_vecs = model_dummy.decode(src_encodings, src_sent_masks, decoder_init_vec, tgt_sents_var[:-1])

assert att_vecs.shape == torch.Size([7, 2, 256]), f"got wrong shape for src_encodings {att_vecs.shape}"
assert abs(att_vecs[0,0,2] - -0.9456) < 0.001, f"got wrong third entry in decoder_init_cell {att_vecs[0,0,2]}"
assert abs(att_vecs[-1,-1,-2] - 0.9151) < 0.001, f"got wrong second to last entry in decoder_init_cell {att_vecs[-1,-1,-2]}"

print("All tests passed!")

### 2.6 Label Smoothing Loss [Programming] (6 points)

The label smoothing loss is a regularization technique used to prevent the model from overfitting to the training data. The label smoothing loss is implemented in the `LabelSmoothingLoss` class in the `utils.py` file.

In [ ]:
from utils import LabelSmoothingLoss
num_labels = 1000
smoothingloss = LabelSmoothingLoss(0.1,tgt_vocab_size=num_labels, padding_idx=0)
output_log_probs = torch.arange(num_labels * 2, dtype=float).reshape(-1, num_labels).log_softmax(dim=-1) # 1 batch
targets = torch.zeros_like(output_log_probs).long()
targets[0, -2] = 1
targets[1, 0] = 1
targets = targets.argmax(dim=-1)
out = smoothingloss.forward(output_log_probs, targets)

print(out)
print("Expected torch tensor of:", [-50.2929, 0])

print("All tests passed!")

### 2.7. Model Training [Non-Programming]

In this section, you will train the model using the given german-english translation dataset. This section may require you to use a GPU. You can choose to use the free tier of Google Colab to train the model. You can refer to the following link to get started with Google Colab - https://colab.research.google.com/notebooks/intro.ipynb. Alternatively, you can subscribe to the paid version of Google Colab or use any other cloud service that provides GPU support.

At the end of this, save the best model you get to carry out further experiments.

You are already provided with training and validation loops for this, you don't need to implement them.

Note - Credits for this section are distributed entirely in the BLEU Score calculation step, as it just needs hyperparameter tuning.

In [49]:
# Hyperparameter Tuning (Feel free to change these). Retain your best performing model.

BATCH_SIZE = 64
embed_dim = 256
hidden_dim = 256
dropout = 0.2
label_smoothing = 0.1
learning_rate = 0.001
n_epochs = 30

In [ ]:
model = NMT(embed_size=embed_dim,
            hidden_size=hidden_dim,
            dropout_rate=dropout,
            label_smoothing=label_smoothing,
            vocab=vocab)
model.train()

In [ ]:
# DO NOT CHANGE THIS CODE BLOCK

uniform_init = 0.1
if np.abs(uniform_init) > 0.:
    print('uniformly initialize parameters [-%f, +%f]' % (uniform_init, uniform_init), file=sys.stderr)
    for p in model.parameters():
        p.data.uniform_(-uniform_init, uniform_init)

vocab_mask = torch.ones(len(vocab.tgt))
vocab_mask[vocab.tgt['<pad>']] = 0

print('Use device: %s' % device, file=sys.stderr)

In [52]:
# DO NOT CHANGE THIS CODE BLOCK

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train(model=model, optimizer=optimizer, train_data=train_data, val_data=valid_data, max_epoch=n_epochs, train_batch_size=BATCH_SIZE, clip_grad=5.0, model_save_path='./model.bin', device=device)

### 2.8. Beam Search [Programming] (3 points)

Beam search is an advanced strategy for generating text sequences, striking a balance between the brute-force exploration of all possible sequences and the narrow focus of greedy sampling. At each step in generating a sequence, given a context $S = \{s_1, s_2, \ldots, s_{n-1}\}$, the algorithm maintains a set of $b$ candidate sequences, called "beams", each with its own cumulative probability. The beam width $b$ controls the number of sequences explored in parallel, with each step extending each candidate sequence by one word and keeping only the top $b$ sequences according to their cumulative probabilities:

$$
B_n = \text{Top}_b \left\{ B_{n-1} \oplus v_i : P(S \oplus v_i|S) \right\}
$$

where $B_n$ represents the set of beams at step $n$, $\oplus$ denotes sequence concatenation, and $\text{Top}_b$ selects the $b$ sequences with the highest cumulative probability. The probability of a sequence is typically the product of the probabilities of its constituent words, adjusted for sequence length to prevent bias toward shorter sequences. Beam search continues this process until a stopping condition is reached, such as all beams ending with an end-of-sequence token or reaching a predetermined sequence length.

While beam search significantly improves the likelihood of finding high-quality sequences by considering multiple hypotheses, its computational cost increases with the beam width. Moreover, despite exploring more options than greedy sampling, beam search can still favor shorter sequences due to its cumulative probability calculation and may converge to similar sequences within the beams, reducing diversity. Nevertheless, it remains a popular choice in NLP tasks where the balance between output quality and computational efficiency is crucial, offering a pragmatic compromise between exhaustive search and deterministic selection.

Beam search is used in sampling too. You can refer to the article in the Sampling section of this assignment for more details on beam search.

You can also refer this video - https://www.youtube.com/watch?v=RLWuzLLSIgw

Note - This implementation will also be used at other places, so, the 3 points is not all the credits for this part. Please implement this carefully, its credits are distributed across sections.

The test below only provides very basic level of sanity check for beam search. Because optimal beam search requires the model to be trained and we need beam search to evaluate your model's BLEU score. You will need to implement the beam search in the `NMT` class in the `nmt_model.py` file.

In [57]:
model = NMT.load('../saved_files/model.bin').to(device) # Load your trained here

In [ ]:
# Covers only basic check for beam search
out = model.beam_search(src_sentences[0], beam_size=5, max_decoding_time_step=100)
assert len(out) == 5, "Incorrect: got incorrect number of outputs " + str(len(out)) + " instead of 5"
assert out[0].score >= out[1].score and out[1].score >= out[2].score and out[2].score >= out[3].score and out[3].score >= out[4].score, "Incorrect: got wrong order of scores"

print("All tests passed!")

### 2.9. BLEU Score [Non-Programming] (6 points)

The models will be evaluated using BLEU score. The BLEU score, which stands for Bilingual Evaluation Understudy, is an algorithm used for evaluating the quality of text that has been machine-translated from one natural language to another. The essence of the BLEU score is to measure how close machine translations are to professional human translations.

This will be a combination of your model training and BLEU score implementation.

Following are the partial credits for BLEU Score -
- bleu < 10.0 - 1 point
- 10.0 <= bleu < 12.0 - 2 point
- 12.0 <= bleu < 15.0 - 3 points
- 15.0 <= bleu < 20.0 - 4 points
- 20.0 <= bleu < 25.0 - 5 points
- bleu > 25.0 - 6 points

In [15]:
beam_size = 2
max_decoding_time_step = 70

In [ ]:
decode(model, beam_size=beam_size, max_decoding_time_step=max_decoding_time_step, device=device)

### 2.10. Label Smoothing Experiments [Non-Programming] (6 points)

In the section below, add as many code cells (code cells only) and train a model with
1. Label Smoothing = 0.0
2. Label Smoothing = 0.2

Report the BLEU score for both the scenarios using Beam Search (you can use the decode function defined before for decoding after training the model)
Feel free to save the weights

Note - You don't need to write model class here. Just declare the NMT model, train your model and evaluate as done above. Preserve all outputs for credits.

In [ ]:
# YOUR IMPLEMENTATION HERE

In [ ]:
# USE THIS CODE CELL TO SAVE YOUR MODEL WEIGHTS

model.save('./model.bin')

## 3. Sampling [Programming + Non-Programming] (34 points)

Sampling refers to the method by which a model selects subsequent words to construct a sentence, based on a probability distribution over the entire vocabulary. This distribution reflects the model's learned understanding of language, predicting the likelihood of each word being the appropriate follow-up in a given context. The choice of sampling strategy is crucial, as it directly influences the balance between creativity and coherence in the generated text. Different strategies navigate this balance in unique ways, impacting the model's output in terms of variability, unpredictability, and fidelity to human-like language patterns.

In the following section, we will explore with the following sampling techniques -
1. Random Sampling
2. Top-p Sampling
3. Top-k Sampling
4. Greedy Sampling
4. Beam Search (already implemented above)

Note - All sampling (except Greedy) have 2 parts - one is a forward pass check that will give you 3 points in the autograder, and one is a BLEU score check that will give you 3 points if your BLEU score crosses the provided threshold. This will be evaluated in non-programming part.

In [8]:
model = NMT.load('../saved_files/model.bin').to(device) # Replace by the file path of your best model here if needed
model.eval()
 
src_sentences = read_corpus(valid_src, source='src')
tgt_sentences = read_corpus(valid_tgt, source='tgt')
 
zipped_lists = sorted(zip(src_sentences, tgt_sentences), key=lambda pair: len(pair[0]), reverse=True)
src_sentences, tgt_sentences = zip(*zipped_lists)

In [9]:
BATCH_SIZE_2 = 256 # If you run out of memory/crash kernel while computing BLEU Score of Sampling, reduce this number. This will increase compute time but decrease memory usage.

### 3.1. Random Sampling [Programming + Non-Programming] (3 + 3 = 6 points)

Random sampling, the most straightforward approach, selects the next word purely based on its predicted probability, allowing for a high degree of diversity but at the risk of producing less coherent sentences. It involves selecting the next word in a sequence based on a probability distribution over the vocabulary. This distribution is derived from the model's predictions, indicating how likely each word is to follow the given context. Here's a mathematical breakdown of how random sampling works:

Given a sequence of words $S = \{s_1, s_2, \ldots, s_{n-1}\}$, an NLP model predicts the next word $s_n$ by estimating the probability distribution $P(V|S)$ over the vocabulary $V = \{v_1, v_2, \ldots, v_M\}$, where $M$ is the size of the vocabulary. The model outputs a probability $P(v_i|S)$ for each word $v_i$ in the vocabulary, indicating the likelihood of $v_i$ being the next word. These probabilities are normalized so that they sum up to 1:

$$\sum_{i=1}^{M} P(v_i|S) = 1$$

Random Sampling involves selecting the next word $s_n$ from the vocabulary $V$ based on the predicted probabilities. Specifically, each word $v_i$ is chosen with a probability $P(v_i|S)$. This selection process can be conceptualized as a random variable $X$ with a multinomial distribution, where the probability mass function (PMF) for $X = v_i$ (the event of choosing word $v_i$ as the next word) is given by:

$$P(X = v_i) = P(v_i|S)$$

This process is repeated for each new word in the sequence, with the context $S$ being updated to include the newly chosen words, until a termination condition is met (e.g., a maximum sequence length or the selection of a special end-of-sequence token).

In the file nmt_model.py, implement Random sampling inside the sample method.

Note - Random sampling will be implemented when both k=None and p=None in the function parameters.






### 3.1.1. Unit Tests for Random Sampling [Programming] (3 points)

In [ ]:
def softmax_fn(logits, dim=None):
    return torch.nn.functional.softmax(torch.tensor([100,-100,-100,-100,-100]).float().reshape(1, -1).expand(logits.size(0),-1).to(device), dim=dim)

out = model.sample(src_sentences[:1], max_decoding_time_step=20, softmax_fn=softmax_fn)
assert len(out) == 1, "Incorrect: got incorrect number of outputs " + str(len(out)) + " instead of 1"
assert len(out[0]) == 5, "Incorrect: got incorrect sample size of " + str(len(out)) + " instead of 5"
out = [vocab.tgt.words2indices(o[0].value) for o in out][0]
assert set(out).__len__() == 2, "Incorrect: got other than 2 unique samples" + str(set(out).__len__())
assert set(out).difference({0, 1}) == set(), "Incorrect: got wrong samples: " + str(set(out))
assert set(out).intersection({1}) == {1}, "Incorrect: start of sequence not in samples"
print('All tests passed!')

### 3.1.2. BLEU Score using Random Sampling [Non-Programming] (3 points)

Adjust the hyperparameters below to get the best BLEU score.

Threshold for points -
- bleu < 10.0 - 0 points
- 10.0 <= bleu <14 - 1 point 
- bleu >= 14.0 - 3 points

In [67]:
max_decoding_time_step = 50 # You can change this for improved results

In [ ]:
# DO NOT CHANGE THIS CODE BLOCK
out = []

N = len(src_sentences) // BATCH_SIZE_2
for b in tqdm(range(N)):
  out.extend(model.sample(sorted(src_sentences[b*BATCH_SIZE_2 : (b+1)*BATCH_SIZE_2], key=lambda x: -len(x)), max_decoding_time_step=max_decoding_time_step, sample_size=2))
out.extend(model.sample(sorted(src_sentences[(N)*BATCH_SIZE_2:], key=lambda x: -len(x)), max_decoding_time_step=max_decoding_time_step, sample_size=2))
best_hyps = []
for i in out:
    best_hyps.append(i[0])
bleu = compute_corpus_level_bleu_score(tgt_sentences, best_hyps)
print('BLEU Score with Random Sampling -', bleu)

### 3.2. Top-k Sampling [Programming + Non-Programming] (3 + 3 = 6 points)

Top-k sampling is a technique employed in text generation tasks to enhance the selection process of the next word in a sequence. Given a context sequence $S = \{s_1, s_2, \ldots, s_{n-1}\}$, and a vocabulary $V = \{v_1, v_2, \ldots, v_M\}$, the model computes a probability distribution $P(V|S)$, predicting the likelihood of each word in the vocabulary being the next appropriate word.

In top-k sampling, the model first identifies the subset $V_k \subseteq V$ consisting of the top $k$ words with the highest probabilities. Mathematically, this subset is defined as:

$$
V_k = \{v | P(v|S) \text{ is among the top } k \text{ of all } P(v_i|S), \forall v_i \in V\}
$$

The probability distribution is then modified to focus solely on these $k$ words, with probabilities outside this set being set to zero. The modified distribution $P'(V|S)$ is defined as:

$$
P'(v_i|S) = 
\begin{cases} 
P(v_i|S) & \text{if } v_i \in V_k \\
0 & \text{otherwise}
\end{cases}
$$

The probabilities in $P'(V|S)$ are then renormalized so that they sum to 1, ensuring a valid probability distribution. The next word is sampled from this adjusted distribution, effectively narrowing the selection to the most probable candidates and discarding less likely, potentially noisy options.

The selection of the next word $s_n$ is, therefore, a random variable $X$ with a distribution reflecting the adjusted probabilities:

$$
P(X = v_i) = \frac{P'(v_i|S)}{\sum_{v_j \in V_k} P'(v_j|S)}
$$

Top-k sampling balances the diversity of text generation with the coherence of the generated sequences, making it a preferred choice in scenarios where quality and readability of the generated text are crucial.

In the file nmt_model.py, enhance the sample method to include top-k sampling with the provided non-None value of $k$.


### 3.2.1. Unit Tests for Top-k Sampling [Programming] (3 points)

In [ ]:
def softmax_fn(logits, dim=None):
    return torch.nn.functional.softmax(torch.tensor([10,10,-10,10,10,-10,-10,-10]).float().reshape(1, -1).expand(logits.size(0),-1).to(device), dim=dim)

out = model.sample(src_sentences[:1], max_decoding_time_step=40, k=4, softmax_fn=softmax_fn)
assert len(out) == 1, "Incorrect: got incorrect number of outputs " + str(len(out)) + " instead of 1"
assert len(out[0]) == 5, "Incorrect: got incorrect sample size of " + str(len(out)) + " instead of 5"
out = [vocab.tgt.words2indices(o[0].value) for o in out][0]
assert set(out).__len__() == 4, "Incorrect: got other than 4 unique samples"
assert set(out).difference({0, 1, 3, 4}) == set(), "Incorrect: got wrong samples: " + str(set(out))
assert set(out).intersection({1}) == {1}, "Incorrect: start of sequence not in samples"
print('All tests passed!')

### 3.2.2. BLEU Score using Top-k Sampling [Non-Programming] (3 points)

Adjust the hyperparameters below to get the best BLEU score.

Threshold for points -
- bleu < 12.0 - 0 points
- 12.0 <= bleu <15.0 - 1 point 
- 15.0 <= bleu < 18.0 - 2 points
- bleu >= 18.0 - 3 points

In [77]:
top_k_value = 4 # You can change this for improved results
max_decoding_time_step = 50 # You can change this for improved results

In [ ]:
# DO NOT CHANGE THIS CODE BLOCK
out=[]
N = len(src_sentences) // BATCH_SIZE_2
for b in tqdm(range(N)):
  out.extend(model.sample(sorted(src_sentences[b*BATCH_SIZE_2 : (b+1)*BATCH_SIZE_2], key=lambda x: -len(x)), max_decoding_time_step=max_decoding_time_step, sample_size=2, k=top_k_value))
out.extend(model.sample(sorted(src_sentences[(N)*BATCH_SIZE_2:], key=lambda x: -len(x)), max_decoding_time_step=max_decoding_time_step, sample_size=2, k=top_k_value))

best_hyps = []
for i in out:
    best_hyps.append(i[0])
bleu = compute_corpus_level_bleu_score(tgt_sentences, best_hyps)
print('BLEU Score with Top-K Sampling -', bleu)

### 3.3. Nucleus/Top-p Sampling [Programming + Non-Programming] (3 + 3 = 6 points)

Nucleus (top-p) sampling is an adaptive technique used in natural language processing (NLP) for generating text sequences that balances the trade-off between randomness and determinism. Unlike top-k sampling that selects the top $k$ most probable words, top-p sampling dynamically selects a subset of the vocabulary by choosing the smallest set of words whose cumulative probability exceeds a threshold $p$. This method focuses on the "nucleus" of probable words, discarding the tail of less likely words to ensure both diversity and coherence in the generated text.

Given a sequence $S = \{s_1, s_2, \ldots, s_{n-1}\}$ and a vocabulary $V = \{v_1, v_2, \ldots, v_M\}$, the model first computes the probability distribution $P(V|S)$ for the next word. The words in $V$ are then sorted by their probability in descending order, resulting in a sorted list $V_{sorted} = \{v_{(1)}, v_{(2)}, \ldots, v_{(M)}\}$ where $P(v_{(1)}|S) \ge P(v_{(2)}|S) \ge \ldots \ge P(v_{(M)}|S)$.

To determine the subset $V_p \subseteq V$ for sampling, we find the smallest $k$ such that:

$$
\sum_{i=1}^{k} P(v_{(i)}|S) > p
$$

The cumulative probability distribution is then adjusted to zero out the probabilities of words not included in $V_p$, and the remaining probabilities are renormalized to sum to 1. The next word $s_n$ is sampled from this adjusted distribution.

The selection of $s_n$ can be represented as sampling from a random variable $X$ with the modified distribution:

$$
P'(X = v_{(i)}) = \frac{P(v_{(i)}|S)}{\sum_{j=1}^{k} P(v_{(j)}|S)}, \text{ for } i \le k
$$

Nucleus sampling's dynamic nature allows it to automatically adjust the breadth of the sampling space based on the context, making it particularly effective for generating high-quality and varied text outputs. This method has become a popular choice in state-of-the-art NLP models for tasks such as text completion, storytelling, and dialogue generation.

In the file nmt_model.py, enhance the sample method to include top-p sampling with the provided non-None value of $p$.



### 3.3.1. Unit Tests for Top-p Sampling [Programming] (3 points)

In [ ]:
def softmax_fn(logits, dim=None):
    # return torch.nn.functional.softmax(logits, dim=dim)
    return torch.nn.functional.softmax(torch.tensor([1,2,3,4,5,6,7,8,9,10]).float().reshape(1, -1).expand(logits.size(0),-1).to(device), dim=dim)

out = model.sample(src_sentences[:1], max_decoding_time_step=90, p=.95, softmax_fn=softmax_fn)
assert len(out) == 1, "Incorrect: got incorrect number of outputs " + str(len(out)) + " instead of 1"
assert len(out[0]) == 5, "Incorrect: got incorrect sample size of " + str(len(out)) + " instead of 5"
out = [vocab.tgt.words2indices(o[0].value) for o in out][0]
assert set(out).__len__() == 4, "Incorrect: got other than 4 unique samples"
assert set(out).difference({8, 1, 9, 7}) == set(), "Incorrect: got wrong samples: " + str(set(out))
assert set(out).intersection({1}) == {1}, "Incorrect: start of sequence not in samples"
print('All tests passed!')

### 3.3.2. BLEU Score using Top-p Sampling [Non-Programming] (3 points)

Adjust the hyperparameters below to get the best BLEU score.

Threshold for points -
- bleu < 12.0 - 0 points
- 12.0 <= bleu <15.0 - 1 point 
- 15.0 <= bleu < 18.0 - 2 points
- bleu >= 18.0 - 3 points

In [11]:
top_p_value = 0.8 # You can change this for improved results
max_decoding_time_step = 50 # You can change this for improved results

In [ ]:
# DO NOT CHANGE THIS CODE BLOCK
out=[]
N = len(src_sentences) // BATCH_SIZE_2
for b in tqdm(range(N)):
  out.extend(model.sample(sorted(src_sentences[b*BATCH_SIZE_2 : (b+1)*BATCH_SIZE_2], key=lambda x: -len(x)), max_decoding_time_step=max_decoding_time_step, sample_size=2, p=top_p_value))
out.extend(model.sample(sorted(src_sentences[(N)*BATCH_SIZE_2:], key=lambda x: -len(x)), max_decoding_time_step=max_decoding_time_step, sample_size=2, p=top_p_value))

best_hyps = []
for i in out:
    best_hyps.append(i[0])
bleu = compute_corpus_level_bleu_score(tgt_sentences, best_hyps)
print('BLEU Score with Top-P/Nucleus Sampling -', bleu)

### 3.4. Greedy Sampling [Non-Programming] (4 points)

Greedy sampling is a deterministic approach in the generation of text sequences from NLP models. At each step of the sequence generation, given a context sequence $S = \{s_1, s_2, \ldots, s_{n-1}\}$, the model computes a probability distribution $P(V|S)$ over the vocabulary $V = \{v_1, v_2, \ldots, v_M\}$. The method then selects the word $v_i$ with the highest probability as the next word in the sequence:

$$
s_n = \arg\max_{v_i \in V} P(v_i|S)
$$

This process is repeated iteratively, updating the context $S$ at each step by appending the newly selected word, until a stopping criterion is met, such as reaching a maximum sequence length or encountering an end-of-sequence token.

Greedy sampling is characterized by its simplicity and efficiency, as it avoids the computational overhead associated with evaluating and sampling from complex probability distributions. However, this simplicity comes at the cost of diversity. The generated sequences may lack variability and could suffer from issues like repetition or predictability. Furthermore, by always selecting the most probable word, greedy sampling can miss out on potentially interesting or more contextually appropriate choices that a stochastic sampling method might explore.

Despite these limitations, greedy sampling is widely used in scenarios where the highest probability sequence is desired, and computational efficiency is paramount. It is particularly common in tasks where a single, clear output is preferred over a diverse set of plausible outputs.


We can directly use argmax of probability distribution to implement greedy sampling. However, in this assignment we will re-use our top-k and top-p sampling to achieve greedy sampling.

Note - The key thing that you get with greedy sampling is repeatability. You always will get the same output as the maximum probability token will be sampled always. 



#### 3.4.1. Greedy Sampling using top-k Sampling [Non-programming] (2 points)

Describe how will you use your top-k sampling code for greedy sampling. In the following code cell, use the parameters of sample method and k (keep p=None) to implement greedy sampling. Show using an example that your output is repeatable.

Also explain the logic behind your implementation.

In [ ]:
out = model.sample(src_sentences, max_decoding_time_step=100, k=1)
for i in out[0]:
    print(i.value)

 YOUR EXPLANATION HERE:

#### 3.4.2. Greedy Sampling using Top-p Sampling [Non-programming] (2 points)

Describe how will you use your top-p sampling code for greedy sampling. In the following code cell, use the parameters of sample method and p (keep k=None) to implement greedy sampling. Show using an example that your output is repeatable.

Also explain the logic behind your implementation.

In [ ]:
# YOUR CODE HERE

YOUR EXPLANATION HERE:

### 3.6. Analysis - Distribution of 'k' in top-p Sampling [Non-programming] (6 points)

Extend and reprogram the implementation of top-p sampling to identify the distribution of number of samples (k) when using 
1. p = 0.5
2. p= 0.8
3. p = 0.9

Write your code below and print the mean and standard deviation of k in all the three cases. Use your existing trained model for this analysis.

Note - Your results should execute from the code, and not just written in the form of markdown cells. You can add more code cells if you want.

Also, in a markdown cell, provide intuitive reasoning why the output is coming that way.

In [ ]:
# YOUR CODE HERE
def k_distribution_using_top_p(p=None):
    mean = None
    std = None
    # YOUR CODE HERE

    # END YOUR CODE
    return mean, std

In [ ]:
# THis cell should print the outputs
mean_p1, std_p1 = k_distribution_using_top_p(0.5)
mean_p2, std_p2 = k_distribution_using_top_p(0.8)
mean_p3, std_p3 = k_distribution_using_top_p(0.9)

print(f'Mean and Standard Deviation for p=0.5: {mean_p1}, {std_p1}')
print(f'Mean and Standard Deviation for p=0.8: {mean_p2}, {std_p2}')
print(f'Mean and Standard Deviation for p=0.9: {mean_p3}, {std_p3}')

YOUR EXPLANATION HERE:


### 3.7. Analysis - Distribution of 'p' in top-k Sampling [Non-programming] (6 points)

Extend and reprogram the implementation of top-k sampling to identify the distribution of cumulative probability of the samples (p) filtered when using 
1. k = 1
2. k = 3
3. k = 5

Write your code below and print the mean and standard deviation of p in all the three cases. Use your existing trained model for this analysis.

Note - Your results should execute from the code, and not just written in the form of markdown cells. You can add more code cells if you want.

Also, in a markdown cell, provide intuitive reasoning why the output is coming that way.


In [29]:
def p_distribution_using_top_k(k=None):
    mean = None
    std = None
    # YOUR CODE HERE

    # END YOUR CODE
    return mean, std

In [ ]:
mean_k1, std_k1 = p_distribution_using_top_k(k=1)
mean_k2, std_k2 = p_distribution_using_top_k(k=3)
mean_k3, std_k3 = p_distribution_using_top_k(k=5)

print(f'Mean and Standard Deviation for k=1: {mean_k1}, {std_k1}')
print(f'Mean and Standard Deviation for k=3: {mean_k2}, {std_k2}')
print(f'Mean and Standard Deviation for k=5: {mean_k3}, {std_k3}')

YOUR EXPLANATION HERE:


## 4. Bonus for all - Try Different Label Smoothing Techniques [Non-Programming] (15 points)

Modify your label smoothing class to try different Label Smoothing techniques and improve your output on beam search. Show the following -

- complete implementation of the Label Smoothing technique
- final BLEU score after training the model on this label smoothing technique
- A write-up describing your design, why you think it works, and how improved it is compared to the existing Label Smoothing technique used for previous sections

For full credits, your technique should give an improved BLEU score over the current one.

Note - You can add as many number of code or markdown cells as you want to provide your solution. Changing noise levels on existing implementation will not count towards any credits.

In [ ]:
# START CODING HERE

YOUR EXPLANATION HERE

## 5. Submission

For programming, submit the following to the Programming Gradescope assignment -
- nmt_model.py
- utils.py
- vocab.json
- vocab.py
- the weights of your best trained model (in Section 2)

Please note, 100 MB is the limit of gradescope submission. Your submission should be less than that

For non-programming, export this notebook as PDF and submit it in the Non-Programming Gradescope assignment. Please label all the pages of your questions correctly and remove any extra print statements that you added that make things redundant and is not required for the evaluation. 